## Rydberg addressing calcs
Book-keeping style calculations to deduce what frequencies for AOMs, cavity(s), etc are needed to address a given Rydberg state

In [2]:
from physconsts import *
from rbconsts import *
from numpy import *
from sympy.physics.wigner import wigner_6j,wigner_3j,clebsch_gordan
from arc import *

In [3]:
qq = ee**2/(4*pi*e0)
rb = Rubidium87()

In [19]:
# all frequencies below are in Hz not s^-1

# ladder states
n1 = 5
l1 = 0
j1 = 1/2
mj1 = 1/2
n2 = 5
l2 = 1
j2 = 3/2
mj2 = 3/2
n3 = 84
l3 = 2
j3 = 5/2
mj3 = 5/2
q = 1

# addressing beam intensities
redPower = 6e-6
bluePower = 0.017
redInt = redPower/(pi*(6e-6)*(8e-6))
blueInt = bluePower/(pi*(4e-6)**2)

# Rabi frequencies
redRabi = sqrt(2*j1+1)*rb.getDipoleMatrixElement(n1,l1,j1,mj1,n2,l2,j2,mj2,q)*ee*a0*sqrt(redInt)/hbar
blueRabi = sqrt(2*j2+1)*rb.getDipoleMatrixElement(n2,l2,j2,mj2,n3,l3,j3,mj3,q)*ee*a0*sqrt(blueInt)/hbar

# hyperfine correction to level c.o.m. energies
f_red_hf1 = 2.563e9 # frequency of 5S1/2,F=2 w.r.t. C.O.M. 
f_red_hf2 = 193.7408e6 # frequency of 5P3/2,F=3 w.r.t. level C.O.M.

# raw transition frequencies on two-step ladder scheme
f_red_transition = rb.getTransitionFrequency(n1,l1,j1,n2,l2,j2) - f_red_hf1 + f_red_hf2
f_blue_transition = rb.getTransitionFrequency(n2,l2,j2,n3,l3,j3) - f_red_hf2

# detuning
f_delta_red = -2.1e9 # approx. 
f_delta_blue = -f_delta_red

# cavity modes
f_780A_cav = 384.2254e12
f_960_cav = 312.6574e12 

# pointing AOMs
f_red_point = 160e6 # +1st order, region 2
f_blue_point = -160e6 # -1st order, region 1

# ponderomotive ("free" electron) Stark shifts
f_ac_red = ee**2*redInt/(4*me*e0*c*(2*pi*(f_red_transition+f_delta_red)**2))
f_ac_blue = ee**2*blueInt/(4*me*e0*c*(2*pi*(f_blue_transition+f_delta_blue)**2))

# calculate necessary remaining corrections for each laser.
# first order: no external field corrections e.g. Zeeman
# all freq. in Hz. at least MHz precision. 
f_780_dp = f_red_transition + f_ac_red - (f_delta_red + f_red_point + f_780A_cav)
f_960_preshift = (f_blue_transition + f_ac_blue - (f_delta_blue + f_blue_point + 2*f_960_cav))/2
print(f"780 double pass={f_780_dp/1e6}MHz")
print(f"960 cavity pre-shift={f_960_preshift/1e6}MHz")

780 double pass=4512.6684291875MHz
960 cavity pre-shift=-1970.352980375MHz


In [17]:
f_ac_red

1.1384381338969606e-31

In [4]:
help(rb.getTransitionFrequency)

Help on method getTransitionFrequency in module arc.alkali_atom_functions:

getTransitionFrequency(n1, l1, j1, n2, l2, j2) method of arc.alkali_atom_data.Rubidium87 instance
    Calculated transition frequency in Hz
    
    Returned values is given relative to the centre of gravity of the
    hyperfine-split states.
    
    Args:
        n1 (int): principal quantum number of the state **from** which we are going
        l1 (int): orbital angular momentum of the state **from** which we are going
        j1 (float): total angular momentum of the state **from** which we are going
        n2 (int): principal quantum number of the state **to** which we are going
        l2 (int): orbital angular momentum of the state **to** which we are going
        j2 (float): total angular momentum of the state **to** which we are going
    
    Returns:
        float:
            transition frequency (in Hz). If the returned value is negative,
            level from which we are going is **above** the